# PROYECTO PLN

## Paula Cabrera, Bartolomé Sabater, Andrea Cardete y Alberto Belmonte

### Importamos los dataframes

In [2]:
import pandas as pd

df = pd.read_csv('sem_eval_train_es.csv')
df2=pd.read_csv('sem_eval_test_grupo_11.csv')

df2

FileNotFoundError: [Errno 2] No such file or directory: 'sem_eval_train_es.csv'

### Eliminamos las filas del dataframe donde el Tweet no tiene sentimientos, es decir, todas las columnas referidas a sentimientos tienen un valor de False

In [157]:
x = df.select_dtypes(include='bool')
for i in x.index: # filas
    y = False
    for j in ['anger','anticipation','disgust','fear','joy','love','optimism',
              'pessimism','sadness','surprise','trust']: # Recorremos cada sentimiento
        if x[j][i]:
            y = True
        elif not y:
            if j == 'trust':
                df = df.drop(i, axis=0)
df

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-Es-01643,@aliciaenp Ajajjaa somos del clan twitteras pe...,False,False,False,False,True,False,False,False,False,False,False
1,2018-Es-05142,@AwadaNai la mala suerte del gato fichame la c...,False,False,False,True,False,False,False,True,False,False,False
2,2018-Es-05379,@audiomano A mí tampoco me agrado mucho eso. E...,True,False,False,False,False,False,False,False,False,False,False
3,2018-Es-00208,Para llevar a los bebes de un lugar a otro deb...,False,False,False,False,True,False,False,False,False,False,False
4,2018-Es-01385,@DalasReview me encanta la terrible hipocresia...,True,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3556,2018-Es-06340,Ahorita quisiera que mi preocupación más grand...,False,False,False,True,False,False,False,True,True,False,False
3557,2018-Es-00439,El mayor criminal del país diciéndole “delincu...,True,False,True,False,False,False,False,False,False,False,False
3558,2018-Es-04919,Mi prima de 4 años se ha enfadado conmigo porq...,True,False,False,False,False,False,False,False,False,True,False
3559,2018-Es-02703,@lennycia Jajaja... Ya seee,False,False,False,False,True,False,False,False,False,False,False


### Obtenemos los valores de los sentimientos y los guardamos en la variable categories

In [158]:
categories=x.columns.values
categories

array(['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love',
       'optimism', 'pessimism', 'sadness', 'surprise', 'trust'],
      dtype=object)

### Realizamos un recuento de las veces que sale cada sentimiento a modo informativo

In [159]:
count = list() # Esto es la lista con el recuento de cada sentimiento
for i in ['anger','anticipation','disgust','fear','joy','love','optimism','pessimism','sadness','surprise','trust']: # Recorremos cada sentimiento
    sum = 0
    for j in df.index: # filas
        if df[i][j]:
            sum += 1
    count.append((i,sum)) # Guardamos la suma
print(count)

[('anger', 1155), ('anticipation', 415), ('disgust', 521), ('fear', 373), ('joy', 1087), ('love', 261), ('optimism', 378), ('pessimism', 578), ('sadness', 845), ('surprise', 169), ('trust', 175)]


### Importamos todas las librerías que nos pueden ser útiles

In [160]:
import pandas as pd
import numpy as np

from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split 
from sklearn.utils import resample

import spacy

import string
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation 
from gensim.models import LdaModel
import gensim

from sklearn import metrics
from sklearn.metrics import classification_report

from textacy.extract import keyterms as kt

import warnings
warnings.filterwarnings('ignore')

import re

### Creamos el modelo de limpieza de datos

In [161]:
#lista de stop-words específicos de nuestro corpus (aproximación)
nlp = spacy.load("es_core_news_sm")

pattern2 = re.compile('[{}]'.format(re.escape(string.punctuation))) #elimina símbolos de puntuación

def clean_text(text, lemas=False):
    """Limpiamos las menciones y URL del texto. Luego convertimos en tokens
    y eliminamos signos de puntuación.
    Si lemas=True extraemos el lema, si no dejamos en minúsculas solamente.
    Como salida volvemos a convertir los tokens en cadena de texto"""
    text = re.sub(r'@[\w_]+|https?://[\w_./]+', '', text) #elimina menciones y URL
    tokens = nlp(text)
    tokens = [tok.lemma_.lower() if lemas else tok.lower_ for tok in tokens if not tok.is_punct]
    filtered_tokens = [pattern2.sub('', tok) for tok in tokens if len(tok)>2]
    filtered_text = ' '.join(filtered_tokens)
    
    return filtered_text

### Aplicamos la limpieza a los datos, lo hacemos con los Tweets limpios y con los lemas, y los añadimos a nuestro dataframe de entrenamiento

In [162]:
df["limpio"]=df['Tweet'].apply(clean_text)

In [163]:
df["lemas"]=df['Tweet'].apply(clean_text, lemas=True)

### Seleccionamos las columnas referidas a sentimientos

In [164]:
y=df.select_dtypes(include='bool')
y

,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,False,False,False,False,True,False,False,False,False,False,False
1,False,False,False,True,False,False,False,True,False,False,False
2,True,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,False
4,True,False,True,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
3556,False,False,False,True,False,False,False,True,True,False,False
3557,True,False,True,False,False,False,False,False,False,False,False
3558,True,False,False,False,False,False,False,False,False,True,False
3559,False,False,False,False,True,False,False,False,False,False,False


### Dividimos los datos en entrenamiento y test

In [165]:
from sklearn.model_selection import train_test_split

# Split data into training and test sets
# Asignamos un 70% a training y un 30% a test
X_train, X_test, y_train, y_test, X_train_lema, X_test_lema = train_test_split(df['limpio'], 
                                                    y,
                                                    df['lemas'],
                                                    test_size=0.3,
                                                    random_state=0
                                                    )

### Extraemos las características mediante BoW, creamos los modelos, los entrenamos y realizamos las predicciones para comprarlos

In [166]:
from sklearn.feature_extraction.text import CountVectorizer

# aprendemos el modelo CountVectorizer sobre el conjunto de train
vect1 = CountVectorizer()

X_train_vectorized1 = vect1.fit_transform(X_train)
print(X_train_vectorized1.shape)


(2377, 6444)


In [167]:
#MODELO BOW - LR

from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

modelLR1 = LogisticRegression(solver='liblinear', class_weight='balanced')
model1 = MultiOutputClassifier(estimator=modelLR)
#Entrenamos el modelo con el conjunto de train
model1.fit(X_train_vectorized1, y_train)

MultiOutputClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                   solver='liblinear'))

In [168]:
prediccion_train = model1.predict(X_train_vectorized1)

In [169]:
from sklearn.metrics import accuracy_score

print('Exactitud: ', accuracy_score(y_train, prediccion_train))

Exactitud:  0.8754732856541859


In [170]:
X_test_vectorized = vect1.transform(X_test)
X_test_vectorized.shape

(1020, 6444)

In [171]:
prediccion = model1.predict(X_test_vectorized)

In [172]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

print('Exactitud: ', accuracy_score(y_test, prediccion))
print(classification_report(y_test, prediccion))

Exactitud:  0.18529411764705883
              precision    recall  f1-score   support

           0       0.63      0.64      0.63       337
           1       0.33      0.41      0.37       113
           2       0.39      0.34      0.36       158
           3       0.72      0.63      0.67       116
           4       0.70      0.60      0.65       324
           5       0.40      0.39      0.39        77
           6       0.39      0.28      0.33       126
           7       0.35      0.39      0.37       166
           8       0.57      0.49      0.53       249
           9       0.29      0.30      0.29        47
          10       0.20      0.13      0.16        54

   micro avg       0.52      0.48      0.50      1767
   macro avg       0.45      0.42      0.43      1767
weighted avg       0.52      0.48      0.50      1767
 samples avg       0.50      0.51      0.48      1767



In [174]:
#Modelo BoW-NB

from sklearn.naive_bayes import MultinomialNB

NB=MultinomialNB()
NB = MultiOutputClassifier(estimator=NB)
#Entrenem el model
NB.fit(X_train_vectorized1, y_train)

#Realitzem les prediccions
pred=NB.predict(X_test_vectorized)

print('Exactitud: ', accuracy_score(y_test, pred))

print(classification_report(y_test, pred))

Exactitud:  0.1450980392156863
              precision    recall  f1-score   support

           0       0.70      0.42      0.52       337
           1       0.38      0.03      0.05       113
           2       0.56      0.06      0.10       158
           3       0.79      0.13      0.22       116
           4       0.77      0.44      0.56       324
           5       1.00      0.04      0.08        77
           6       0.75      0.02      0.05       126
           7       0.42      0.05      0.09       166
           8       0.74      0.26      0.38       249
           9       0.00      0.00      0.00        47
          10       0.00      0.00      0.00        54

   micro avg       0.71      0.22      0.34      1767
   macro avg       0.55      0.13      0.19      1767
weighted avg       0.64      0.22      0.30      1767
 samples avg       0.34      0.26      0.28      1767



In [175]:
#Modelo BoW-SVM

from sklearn.linear_model import SGDClassifier

SVM=SGDClassifier(loss='hinge', max_iter=10000, tol=1e-5, class_weight='balanced')
SVM = MultiOutputClassifier(estimator=SVM)

#Entrenem el model
SVM.fit(X_train_vectorized1, y_train)

#Realitzem les prediccions
pred=SVM.predict(X_test_vectorized)
print('Exactitud: ', accuracy_score(y_test, pred))
print(classification_report(y_test, pred))

Exactitud:  0.15098039215686274
              precision    recall  f1-score   support

           0       0.61      0.57      0.59       337
           1       0.24      0.47      0.32       113
           2       0.35      0.32      0.33       158
           3       0.67      0.57      0.62       116
           4       0.65      0.60      0.62       324
           5       0.36      0.42      0.38        77
           6       0.37      0.24      0.29       126
           7       0.27      0.30      0.29       166
           8       0.53      0.45      0.48       249
           9       0.24      0.36      0.29        47
          10       0.22      0.15      0.18        54

   micro avg       0.46      0.45      0.46      1767
   macro avg       0.41      0.40      0.40      1767
weighted avg       0.48      0.45      0.46      1767
 samples avg       0.45      0.48      0.43      1767



### Extraemos las características mediante TF-IDF, creamos los modelos, los entrenamos y realizamos las predicciones para comprarlos

In [176]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline

vect = TfidfVectorizer()

modelo = make_pipeline(vect, model)
#Entrenamos el modelo con el conjunto de train
modelo.fit(X_train, y_train)

# Predecimos sobre el conjunto de test
prediccion = modelo.predict(X_test)
print('Exactitud: ', accuracy_score(y_test, prediccion))
print(classification_report(y_test, prediccion))

Exactitud:  0.16666666666666666
              precision    recall  f1-score   support

           0       0.61      0.67      0.64       337
           1       0.29      0.44      0.35       113
           2       0.33      0.39      0.36       158
           3       0.69      0.61      0.65       116
           4       0.68      0.64      0.66       324
           5       0.37      0.40      0.39        77
           6       0.34      0.33      0.34       126
           7       0.34      0.49      0.40       166
           8       0.53      0.53      0.53       249
           9       0.31      0.32      0.31        47
          10       0.20      0.17      0.18        54

   micro avg       0.48      0.52      0.50      1767
   macro avg       0.43      0.45      0.44      1767
weighted avg       0.50      0.52      0.51      1767
 samples avg       0.49      0.55      0.49      1767



In [177]:
modelo = make_pipeline(vect, NB)
#Entrenamos el modelo con el conjunto de train
modelo.fit(X_train, y_train)

# Predecimos sobre el conjunto de test
prediccion = modelo.predict(X_test)
print('Exactitud: ', accuracy_score(y_test, prediccion))
print(classification_report(y_test, prediccion))

Exactitud:  0.04803921568627451
              precision    recall  f1-score   support

           0       0.80      0.12      0.21       337
           1       0.00      0.00      0.00       113
           2       0.00      0.00      0.00       158
           3       0.00      0.00      0.00       116
           4       0.95      0.13      0.22       324
           5       0.00      0.00      0.00        77
           6       0.00      0.00      0.00       126
           7       0.00      0.00      0.00       166
           8       0.92      0.04      0.08       249
           9       0.00      0.00      0.00        47
          10       0.00      0.00      0.00        54

   micro avg       0.88      0.05      0.10      1767
   macro avg       0.24      0.03      0.05      1767
weighted avg       0.46      0.05      0.09      1767
 samples avg       0.09      0.07      0.08      1767



In [178]:
modelo = make_pipeline(vect, SVM)
#Entrenamos el modelo con el conjunto de train
modelo.fit(X_train, y_train)

# Predecimos sobre el conjunto de test
prediccion = modelo.predict(X_test)
print('Exactitud: ', accuracy_score(y_test, prediccion))
print(classification_report(y_test, prediccion))

Exactitud:  0.1676470588235294
              precision    recall  f1-score   support

           0       0.59      0.62      0.61       337
           1       0.33      0.39      0.36       113
           2       0.33      0.33      0.33       158
           3       0.68      0.60      0.64       116
           4       0.67      0.60      0.64       324
           5       0.37      0.38      0.37        77
           6       0.29      0.20      0.24       126
           7       0.32      0.36      0.34       166
           8       0.51      0.49      0.50       249
           9       0.31      0.34      0.32        47
          10       0.22      0.13      0.16        54

   micro avg       0.49      0.47      0.48      1767
   macro avg       0.42      0.40      0.41      1767
weighted avg       0.49      0.47      0.48      1767
 samples avg       0.48      0.50      0.46      1767



### Hacemos los mismo pero con los lemas

In [179]:
#vectorizamos
vect = CountVectorizer()

X_train_vectorized = vect.fit_transform(X_train_lema)
X_test_vectorized = vect.transform(X_test_lema)
X_train_vectorized.shape

(2377, 5337)

In [180]:
#Modelo BoW-LR
model.fit(X_train_vectorized, y_train)
prediccion = model.predict(X_test_vectorized)
print('Exactitud: ', accuracy_score(y_test, prediccion))
print(classification_report(y_test, prediccion))

Exactitud:  0.1715686274509804
              precision    recall  f1-score   support

           0       0.61      0.59      0.60       337
           1       0.32      0.40      0.35       113
           2       0.33      0.32      0.33       158
           3       0.70      0.62      0.66       116
           4       0.69      0.63      0.66       324
           5       0.43      0.43      0.43        77
           6       0.38      0.31      0.34       126
           7       0.33      0.42      0.37       166
           8       0.55      0.51      0.53       249
           9       0.23      0.28      0.25        47
          10       0.20      0.17      0.18        54

   micro avg       0.49      0.49      0.49      1767
   macro avg       0.43      0.43      0.43      1767
weighted avg       0.51      0.49      0.50      1767
 samples avg       0.48      0.51      0.46      1767



In [181]:
#Modelo BoW-NB
NB.fit(X_train_vectorized, y_train)
prediccion = NB.predict(X_test_vectorized)
print('Exactitud: ', accuracy_score(y_test, prediccion))
print(classification_report(y_test, prediccion))

Exactitud:  0.1411764705882353
              precision    recall  f1-score   support

           0       0.72      0.41      0.52       337
           1       0.50      0.02      0.03       113
           2       0.58      0.07      0.12       158
           3       0.73      0.09      0.17       116
           4       0.79      0.36      0.50       324
           5       1.00      0.06      0.12        77
           6       0.50      0.01      0.02       126
           7       0.39      0.04      0.08       166
           8       0.72      0.22      0.34       249
           9       0.00      0.00      0.00        47
          10       0.00      0.00      0.00        54

   micro avg       0.72      0.20      0.31      1767
   macro avg       0.54      0.12      0.17      1767
weighted avg       0.63      0.20      0.28      1767
 samples avg       0.31      0.24      0.26      1767



In [182]:
#Modelo BoW-SVM
SVM.fit(X_train_vectorized, y_train)
prediccion = SVM.predict(X_test_vectorized)
print('Exactitud: ', accuracy_score(y_test, prediccion))
print(classification_report(y_test, prediccion))

Exactitud:  0.14411764705882352
              precision    recall  f1-score   support

           0       0.59      0.61      0.60       337
           1       0.27      0.35      0.30       113
           2       0.27      0.38      0.32       158
           3       0.67      0.55      0.61       116
           4       0.66      0.59      0.62       324
           5       0.38      0.29      0.33        77
           6       0.32      0.32      0.32       126
           7       0.32      0.43      0.36       166
           8       0.47      0.56      0.51       249
           9       0.17      0.26      0.20        47
          10       0.17      0.13      0.15        54

   micro avg       0.44      0.48      0.46      1767
   macro avg       0.39      0.40      0.39      1767
weighted avg       0.46      0.48      0.47      1767
 samples avg       0.44      0.50      0.44      1767



In [183]:
#vectorizamos
vect = TfidfVectorizer()

X_train_vectorized = vect.fit_transform(X_train_lema)
X_test_vectorized = vect.transform(X_test_lema)
X_train_vectorized.shape

(2377, 5337)

In [184]:
#Modelo TFIDF-LR
model.fit(X_train_vectorized, y_train)
prediccion = model.predict(X_test_vectorized)
print('Exactitud: ', accuracy_score(y_test, prediccion))
print(classification_report(y_test, prediccion))

Exactitud:  0.15588235294117647
              precision    recall  f1-score   support

           0       0.59      0.64      0.61       337
           1       0.27      0.46      0.34       113
           2       0.30      0.42      0.35       158
           3       0.67      0.65      0.66       116
           4       0.67      0.65      0.66       324
           5       0.36      0.45      0.40        77
           6       0.32      0.37      0.34       126
           7       0.30      0.50      0.37       166
           8       0.50      0.55      0.52       249
           9       0.21      0.32      0.25        47
          10       0.16      0.17      0.16        54

   micro avg       0.44      0.53      0.48      1767
   macro avg       0.39      0.47      0.43      1767
weighted avg       0.47      0.53      0.50      1767
 samples avg       0.46      0.56      0.47      1767



In [185]:
#Modelo TFIDF-NB
NB.fit(X_train_vectorized, y_train)
prediccion = NB.predict(X_test_vectorized)
print('Exactitud: ', accuracy_score(y_test, prediccion))
print(classification_report(y_test, prediccion))

Exactitud:  0.054901960784313725
              precision    recall  f1-score   support

           0       0.85      0.13      0.23       337
           1       1.00      0.01      0.02       113
           2       0.00      0.00      0.00       158
           3       0.00      0.00      0.00       116
           4       0.96      0.14      0.25       324
           5       0.00      0.00      0.00        77
           6       0.00      0.00      0.00       126
           7       0.00      0.00      0.00       166
           8       1.00      0.04      0.08       249
           9       0.00      0.00      0.00        47
          10       0.00      0.00      0.00        54

   micro avg       0.91      0.06      0.11      1767
   macro avg       0.35      0.03      0.05      1767
weighted avg       0.54      0.06      0.10      1767
 samples avg       0.10      0.08      0.08      1767



In [186]:
#Modelo TFIDF-SVM
SVM.fit(X_train_vectorized, y_train)
prediccion = SVM.predict(X_test_vectorized)
print('Exactitud: ', accuracy_score(y_test, prediccion))
print(classification_report(y_test, prediccion))

Exactitud:  0.15588235294117647
              precision    recall  f1-score   support

           0       0.58      0.60      0.59       337
           1       0.27      0.32      0.29       113
           2       0.29      0.32      0.31       158
           3       0.63      0.55      0.59       116
           4       0.66      0.61      0.63       324
           5       0.45      0.38      0.41        77
           6       0.31      0.28      0.29       126
           7       0.30      0.39      0.34       166
           8       0.50      0.51      0.51       249
           9       0.17      0.19      0.18        47
          10       0.20      0.15      0.17        54

   micro avg       0.46      0.47      0.46      1767
   macro avg       0.40      0.39      0.39      1767
weighted avg       0.47      0.47      0.46      1767
 samples avg       0.45      0.49      0.44      1767



### Una vez realizados todos los modelos y comparando su accuracy y sus tablas, decidimos que el mejor modelo entrenado es el LogisticRegression extrayendo las características mediante BoW, por lo que vamos a realizar la predicción de los datos de test mediante este modelo

### Limpiamos los Tweets del dataframe de test proporcionado para, a continuación, realizar la predicción

In [187]:
tweet=df2['Tweet']
limpio=tweet.apply(clean_text)
limpio

0                                      jaj tan resentido
1                                estoy emocionada aaaaah
2                        satisfacción por deber cumplido
3                considerate afortunada mío hasta ignora
4                                                  enojo
                             ...                        
674                 que nunca vas dejar ponerme nerviosa
675                    anon sos terrible cornuda bla etc
676           problemas trabajo embarque detenido terror
677    nostalgia unas horas lanzamiento oficial secue...
678                    anoche estalló vena furia milagro
Name: Tweet, Length: 679, dtype: object

In [188]:
limpio_vec=vect1.transform(limpio)

In [189]:
print(limpio_vec.shape)
print(X_train_vectorized1.shape)

(679, 6444)
(2377, 6444)


In [190]:
#Modelo BoW-LR

#Realitzem les prediccions
pred=model1.predict(limpio_vec)
pred


array([[ True, False,  True, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False,  True],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ...,  True, False, False],
       [ True, False, False, ..., False, False, False]])

In [191]:
definitivo=pd.concat([df2['ID'], pd.DataFrame(pred,columns=categories)], axis=1)
definitivo.iloc[:,1:]=definitivo.iloc[:,1:]==1
definitivo

,ID,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-Es-00074,True,False,True,False,False,False,False,False,False,False,False
1,2018-Es-02721,False,False,False,False,True,False,False,False,False,False,False
2,2018-Es-04447,False,False,False,False,True,False,False,False,False,False,True
3,2018-Es-04392,False,False,False,False,True,False,False,False,False,False,True
4,2018-Es-03765,True,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
674,2018-Es-00162,False,True,False,True,False,False,False,False,False,False,False
675,2018-Es-05538,True,False,True,False,False,False,False,False,False,False,False
676,2018-Es-00430,False,False,False,True,False,False,False,False,False,False,False
677,2018-Es-02679,False,False,False,False,False,False,False,False,True,False,False


In [192]:
#Juntamos los dataframes para obtener el definitivo
definitivo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679 entries, 0 to 678
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            679 non-null    object
 1   anger         679 non-null    bool  
 2   anticipation  679 non-null    bool  
 3   disgust       679 non-null    bool  
 4   fear          679 non-null    bool  
 5   joy           679 non-null    bool  
 6   love          679 non-null    bool  
 7   optimism      679 non-null    bool  
 8   pessimism     679 non-null    bool  
 9   sadness       679 non-null    bool  
 10  surprise      679 non-null    bool  
 11  trust         679 non-null    bool  
dtypes: bool(11), object(1)
memory usage: 12.7+ KB


### Creamos el csv

In [193]:
definitivo.to_csv('C:/Users/1487846/Documents/GCD/TERCERO/SEGUNDO CUATRI/PLN/grupo11.csv')